# Biomedical Keyword Tagging & Normalization Skeleton

## [1.Data Extraction_&_Preprocessing](#1.Data_Extraction_&_Preprocessing)

### Dataset
### Type of Dataset: Patent Data Abstract: Patbase
### Size of Dataset: 2000k Records

### Keywords extraction and Scispacy
ref:https://betterprogramming.pub/extract-keywords-using-spacy-in-python-4a8415478fbf


### Sentence embedding 
https://huggingface.co/gsarti/scibert-nli


### Preprocessing

1) We used <b>NLTK</b> to tokenize and Lemmatize the Text 

### Improvements

#### Word Tokenization and Lemmatization can be improved with POS Tagging

## [2.Keyword_&_Phrase_Extraction](#2.Keyword_&_Phrase_Extraction)

1) Used <b>Count Vectorizer and Tfidf</b> to get candidate keywords \
2) <b>Paraphrase mining sentence embedding</b> used to get sentence embedding link:https://www.sbert.net/examples/applications/paraphrase-mining/README.html \
3) <b>Maximum Marginal Retrival</b> to check cosine similarity across documents to consolidate candidate terms with terms across documents 

### Improvements 
#### After CV and TFIDF we can apply lemmatization with adding vectors to increase weightage across similar words with same meaning to reduce redundancy. 
#### Encoder Decoder Model with self Attention can be utilized to get better embedding with context and similarity
####  Pretrained BERT can be used to get better contextual similarity 

### [3.Visualization](#3.Visualization)
#### Motivation 
To get View of Textual Meaning and Relation of keyowrds and phrases across documents to get better intuition. 

steps
1) Used <b>KMapper</b> to visualize top keyowrds across documeents \
2) Used <b>DBScan Clustering Algorithm</b> to combine various phrases within documents AND across documents with TFIDF & Ngram

### Conclusion 
1) Here is implementation workflow for Keyphrase Extraction with visualization 
2) We can see there is duplicate and redundant terms which can be 

In [7]:
import string
import re
import numpy as np
import pandas as pd
import codecs
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from scipy import spatial


import nltk
from nltk.corpus import wordnet,stopwords
from nltk.stem import WordNetLemmatizer 
from nltk import WordNetLemmatizer, pos_tag, word_tokenize



from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

from scipy import stats

import spacy
nlp = spacy.load('en_core_sci_sm', disable=['parser', 'ner'])


import tqdm

# doc=nlp(text)

# from gensim.summarization import keywords
stop_words=stopwords.words('english')
lemmatizer = WordNetLemmatizer()
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
from string import punctuation

# 1.Data_Extraction_&_Preprocessing

In [5]:
pat_data=pd.read_csv('../data/patbase_data.csv')
abs_text=pat_data['Abstract']
print(type(abs_text))

# sample_data=pd.read_excel('Keyword_validation.xlsx')
# sample_abs_data=sample_data['Abstract']
# sample_abs_data

<class 'pandas.core.series.Series'>


In [8]:

def text_cleaning(text):
    cleaned_txt = re.sub('<[^<]+>', ' ', str(text))
    cleaned_txt = re.sub( r'\[.*?\]', ' ',cleaned_txt)  #remove brackets
#     cleaned_txt = re.sub('\d+', ' ', cleaned_txt)
#     cleaned_txt = re.sub('\(\w+\)', ' ', cleaned_txt)
#     cleaned_txt=text.translate(str.maketrans(' ', ' ',string.punctuation)) 
    return cleaned_txt
   
def stopword_removal(text):
    cleaned_txt = ' '.join([token for token in word_tokenize(text) if token not in stop_words])
    return cleaned_txt
def lemmatize_text(text):
    cleaned_txt=' '.join([lemmatizer.lemmatize(token) for token in word_tokenize(text)])

    return cleaned_txt

def preprocessed_text(df):
    
    col=['nltk_tokens','nouns_verbs','lemmatize_text']
    preprocess_txt=pd.DataFrame(columns=col)
    abs_clean_text=df.apply(text_cleaning)
    
    stopword_removed_text=abs_clean_text.apply(stopword_removal)         
    preprocess_txt['lemmatize_text']=stopword_removed_text.apply(lemmatize_text)
  
    for ind,text in tqdm(preprocess_txt['lemmatize_text'].iteritems()):       
        tokens=word_tokenize(text)
        pos_tokens=nltk.pos_tag(tokens)
        preprocess_txt.at[ind,'nouns_verbs']=[pos_token for pos_token,pos in pos_tokens if pos.startswith('N' or 'V')]
        preprocess_txt.at[ind,'nltk_tokens']=tokens     
    return preprocess_txt

ERROR! Session/line number was not unique in database. History logging moved to new session 1774


In [9]:
preprocessed_df=preprocessed_text(abs_text)
preprocessed_df

# preprocessed_df1=preprocessed_text(abs_text)
# preprocessed_df1

nltk_tokens  \
0     [The, purpose, present, invention, provide, re...   
1     [A, battery, pack, including, least, one, batt...   
2     [A, quaternary, ammonium, compound, reaction, ...   
3     [A, washing, machine, comprises, outer, tub, ,...   
4     [The, present, invention, relates, ceiling, ty...   
...                                                 ...   
1995  [A, control, method, includes, :, establishing...   
1996  [A, numerical, controller, capable, determinin...   
1997  [A, magnetic, resonance, apparatus, (, 100, ,,...   
1998  [A, fluid-conveying, device, conveying, fluid,...   
1999  [The, present, disclosure, provides, method, t...   

                                            nouns_verbs  \
0     [invention, resin, balance, variety, property,...   
1     [battery, pack, battery, cell, case, accommoda...   
2     [ammonium, compound, reaction, product, weight...   
3     [machine, comprises, tub, tub, pulsator, rotor...   
4     [invention, ceiling, type, air, conditioner, h...   
...                                                 ...   
1995  [control, method, connection, device, device, ...   
1996  [controller, input, value, operator, range, li...   
1997  [resonance, apparatus, resonance, apparatus, p...   
1998  [device, tank, pump, pot, pump, drive, region,...   
1999  [disclosure, wound, healing, subject, method, ...   

                                         lemmatize_text  
0     The purpose present invention provide resin ex...  
1     A battery pack including least one battery cel...  
2     A quaternary ammonium compound reaction produc...  
3     A washing machine comprises outer tub , inner ...  
4     The present invention relates ceiling type air...  
...                                                 ...  
1995  A control method includes : establishing conne...  
1996  A numerical controller capable determining whe...  
1997  A magnetic resonance apparatus ( 100 , 800 , 1...  
1998  A fluid-conveying device conveying fluid tank ...  
1999  The present disclosure provides method treatin...  

[2000 rows x 3 columns]

# 2.Keyword_&_Phrase_Extraction

### keyphrase extraction workflow 

1. Extraction of keywords using count vectorizer,Tfidf,spacy, gensim 
2. Generating Sentence embedding from Pretrained Model for each document
3. Duplicate and Redundant removal with MMR for single document ,Doc_word cosine Simililarity for Multiple document

## Single doc

In [10]:
def cv_word_doc_embedding(doc):
    cv_countvec = CountVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=1).fit([doc])
    cv_words = cv_countvec.get_feature_names()
#     print(cv_words)
    cv_doc_embeddings = embedding_model.encode([doc])
    cv_word_embeddings =embedding_model.encode(cv_words)
    return cv_word_embeddings,cv_doc_embeddings,cv_words
    

In [11]:
def tfidf_word_doc_embedding(doc):
    tfidf_countvec = TfidfVectorizer(ngram_range=(1,2), stop_words=stop_words, min_df=0.8).fit([doc])
    tfidf_words = tfidf_countvec.get_feature_names()    
    print(tfidf_words)
    tfidf_doc_embeddings = embedding_model.encode([doc])
    tfidf_word_embeddings = embedding_model.encode(tfidf_words)
    return tfidf_word_embeddings,tfidf_doc_embeddings,tfidf_words


In [14]:
top_n=10
diversity=0.5
from sentence_transformers import SentenceTransformer,models

# from torch import nn
# # word_embedding_model = models.Transformer('distilbert-base-uncased', max_seq_length=64)
# # pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension()).to('cuda:0')
# # dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=64,
# #                            activation_function=nn.Tanh()).to('cuda:0')
# # embedding_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

# # # embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
embedding_model = SentenceTransformer("gsarti/scibert-nli")



In [11]:
def mmr(doc_embedding: np.ndarray,word_embeddings: np.ndarray,words,top_n,diversity):
    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)
    
    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
    
    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        if mmr.size>0:
            mmr_idx = candidates_idx[np.argmax(mmr)]
        
            # Update keywords & candidates
            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)
            
    return [(words[idx], round((word_doc_similarity.reshape(1, -1)[0][idx]), 4)) for idx in keywords_idx]

In [12]:
# pat_data=pd.read_excel('V:/ML_projects/Merckgroup/Keyword_tagging/data/recent_5k.xlsx')
# abs_text=pat_data['Abstract']
# preprocessed_df1=preprocessed_text(abs_text)
# preprocessed_df1
doc=preprocessed_df['lemmatize_text'][:300]

cv_keywords=[]
tfidf_keywords=[]
cv_phrases=[]
tfidf_phrases=[]
for i in doc:
    cv_word_embed,cv_doc_embed,cv_words=cv_word_doc_embedding(i)
    if cv_words:
        cv_keywords_doc=mmr(cv_doc_embed,cv_word_embed,cv_words,top_n,diversity)
        cv_keywords_doc=','.join([key[0] for key in cv_keywords_doc])
        cv_phrases_doc=''.join([key[0] for key in cv_keywords_doc])
        cv_keywords.append(cv_keywords_doc)
        cv_phrases.append(cv_phrases_doc)
    
    tfidf_word_embed,tfidf_doc_embed,tfidf_words=tfidf_word_doc_embedding(i)
    tfidf_keywords_doc=mmr(tfidf_doc_embed,tfidf_word_embed,tfidf_words,5,0.2)
    tfidf_keywords_doc=','.join([key[0] for key in tfidf_keywords_doc])
    tfidf_phrases_doc=''.join([key[0] for key in tfidf_keywords_doc])
#     print('cv_keywords',cv_keywords_doc)
#     print('cv_phrases',cv_phrases_doc)
 
#     print('tfidf_keywords',tfidf_keywords_doc)
#     print('tfidf_phrases',tfidf_phrases_doc)
    
    tfidf_keywords.append(tfidf_keywords_doc)
    tfidf_phrases.append(tfidf_phrases_doc)

C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarni

['22', '22 38', '38', '38 wherein', '450', '450 nm', '550', '550 nm', '75', '75 le', '93', '93 content', 'aromatic', 'aromatic structure', 'balance', 'balance variety', 'characterized', 'characterized structural', 'constitutes', 'constitutes resin', 'content', 'content aromatic', 'content mass', 'difference', 'difference film', 'difference r450', 'difference r550', 'equal', 'equal 22', 'equal 93', 'equal le', 'excellent', 'excellent balance', 'film', 'film obtained', 'film phase', 'film produced', 'formula', 'formula le', 'heat', 'heat resistance', 'heat stability', 'includes', 'includes aromatic', 'invention', 'invention provide', 'invention relates', 'le', 'le equal', 'mass', 'mass percent', 'mechanical', 'mechanical property', 'nm', 'nm phase', 'nm stretched', 'obtained', 'obtained using', 'optical', 'optical film', 'optical physical', 'percent', 'percent aromatic', 'phase', 'phase difference', 'physical', 'physical property', 'polycondensation', 'polycondensation resin', 'present',

C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarni

['1000', '1000 acid', 'acid', 'acid activated', 'activated', 'activated alkylating', 'agent', 'agent diacid', 'alkylating', 'alkylating agent', 'amine', 'amine molecular', 'ammonium', 'ammonium compound', 'atom', 'carbon', 'carbon atom', 'compound', 'compound reaction', 'diacid', 'diacid including', 'hydrocarbyl', 'hydrocarbyl moiety', 'including', 'including optionally', 'le', 'le 1000', 'least', 'least carbon', 'moiety', 'moiety least', 'molecular', 'molecular weight', 'optionally', 'optionally substituted', 'product', 'product tertiary', 'quaternary', 'quaternary ammonium', 'reaction', 'reaction product', 'substituted', 'substituted hydrocarbyl', 'tertiary', 'tertiary amine', 'weight', 'weight le']
['automatic', 'automatic washing', 'comprises', 'comprises outer', 'consumption', 'consumption stable', 'deceleration', 'deceleration mechanism', 'dehydration', 'dehydration pulsator', 'direction', 'direction dehydration', 'direction speed', 'drive', 'drive inner', 'drive pulsator', 'driv

C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarni

...

['accordingto', 'accordingto system', 'achieved', 'achieved water', 'air', 'air flowing', 'air inlet', 'air velocity', 'applied', 'applied toilet', 'arrangement', 'arrangement check', 'balance', 'balance via', 'better', 'better deodorization', 'blown', 'blown toilet', 'check', 'check valve', 'comprises', 'comprises first', 'connected', 'connected air', 'connected sequence', 'connected three', 'connected via', 'corner', 'corner joint', 'correspondingly', 'correspondingly connected', 'deodorization', 'deodorization effect', 'deodorization system', 'deodorizing', 'deodorizing therefore', 'diameter', 'diameter connected', 'diameter matched', 'different', 'different diameter', 'different inner', 'different path', 'discharge', 'discharge pipe', 'discloses', 'discloses negative', 'effect', 'effect achieved', 'equipment', 'equipment discloses', 'external', 'external corner', 'fan', 'fan accordingto', 'fan discharge', 'fan wherein', 'field', 'field toilet', 'first', 'first discharge', 'first ne

C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['annular', 'annular top', 'area', 'area increase', 'bristle', 'bristle one', 'bristle wall', 'care', 'care implement', 'cavity', 'cavity cavity', 'cavity transverse', 'cleaning', 'cleaning element', 'conical', 'conical tuft', 'coupled', 'coupled handle', 'cross', 'cross sectional', 'defining', 'defining cavity', 'distance', 'distance front', 'element', 'element extend', 'element include', 'embodiment', 'embodiment bristle', 'embodiment head', 'embodiment invention', 'extend', 'extend front', 'four', 'four conical', 'front', 'front surface', 'furthermore', 'furthermore embodiment', 'handle', 'handle head', 'head', 'head coupled', 'head embodiment', 'head front', 'head furthermore', 'head includes', 'head plurality', 'implement', 'implement includes', 'inclined', 'inclined relative', 'include', 'include conical', 'includes', 'includes conical', 'includes handle', 'includes one', 'increase', 'increase distance', 'inner', 'inner surface', 'invention', 'invention oral', 'one', 'one conical

C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarni

['according', 'according place', 'according preset', 'according taxi', 'acquiring', 'acquiring place', 'advance', 'advance taxi', 'appropriate', 'appropriate using', 'art', 'art taxi', 'comprises', 'comprises following', 'computer', 'computer network', 'corresponding', 'corresponding plurality', 'corresponding taxi', 'departure', 'departure destination', 'destination', 'destination determining', 'destination user', 'determining', 'determining taxi', 'discloses', 'discloses taxi', 'displaying', 'displaying plurality', 'experience', 'experience user', 'field', 'field method', 'following', 'following step', 'hailing', 'hailing experience', 'hailing processing', 'hailing prompt', 'hailing request', 'hailing type', 'improved', 'information', 'information corresponding', 'information sent', 'information taxi', 'information user', 'invention', 'invention discloses', 'invention problem', 'method', 'method comprises', 'method system', 'mode', 'mode according', 'network', 'network technology', '

In [13]:
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.pos_ in pos_tag):
            result.append(token.text)
                
    return result # 5

In [14]:
solr_data=pd.read_excel('../data/recent_5k.xlsx')

def text_cleaning(text):
    cleaned_txt = re.sub('<[^<]+>', ' ', str(text))
    cleaned_txt = re.sub( r'\[.*?\]', ' ',cleaned_txt)  #remove brackets
    cleaned_txt = re.sub('\d+', ' ', cleaned_txt)
    cleaned_txt = re.sub('\(\w+\)', ' ', cleaned_txt)
#     cleaned_txt=text.translate(str.maketrans(' ', ' ',string.punctuation)) 
    return cleaned_txt


solr_data['clean_Abstract']=solr_data['Abstract'].apply(text_cleaning)
solr_data['scispacy_tags_extract']=solr_data['clean_Abstract'].apply(get_hotwords)

# for i in tfidf_keywords:
#     print(i)
#     break


In [15]:
solr_data['scispacy_tags_extract']

0       [contamination, meat, antimicrobial-resistant,...
1       [receptor, gpr, attractive, drug, target, type...
2       [structures, melatonin, ferulic, acid, tertiar...
3       [food, drug, administration-approved, treatmen...
4       [productivity, traditional, backyard, poultry,...
                              ...                        
4995    [nutritional, risk, screening, clinical, setti...
4996    [purpose, number, studies, importance, vitamin...
4997    [non-alcoholic-fatty, liver, disease, specific...
4998    [model, food, policy, acceptability, model, le...
4999    [study, physicochemical, characteristics, effe...
Name: scispacy_tags_extract, Length: 5000, dtype: object

In [55]:

# patbase_key_extract=pd.DataFrame()
# patbase_key_extract['Abstract']=abs_text[:100]
# patbase_key_extract['CV_keywords']=cv_keywords
# patbase_key_extract['tfidf_keywords']=tfidf_keywords

In [18]:
# patbase_key_extract.to_csv('../Projects/Keyword_tagging/data/patbase_keyextract.csv')

## Multiple  documents

In [16]:
import numpy as np
import itertools
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Tuple
import nltk
from nltk.corpus import stopwords
# from nltk.tokenize import sentence
from sklearn.feature_extraction.text import TfidfVectorizer


from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")



## Count Vectorizer

In [17]:
def phrase_generator(docs,words,countvec):
    
    top_n:int=10
    diversity:0.5
    df = countvec.transform(docs)
    
    keywords=[]    
    phrases=[]
    
    
    for index, doc in enumerate(docs):
        doc_words = [words[i] for i in df[index].nonzero()[1]]
        if doc_words:
            doc_word_embeddings = np.array([cv_word_embeddings[i] for i in df[index].nonzero()[1]])
            distances = cosine_similarity([cv_doc_embeddings[index]], doc_word_embeddings)[0]
            doc_keywords = [(doc_words[i], round(float(distances[i]), 4)) for i in distances.argsort()[-top_n:]]
            doc_keywords=','.join([key[0] for key in doc_keywords])
            doc_phrases=''.join([key[0] for key in doc_keywords])
            keywords.append(doc_keywords)
            phrases.append(doc_phrases)                                 
    return keywords,phrases
                              

In [21]:
docs=preprocessed_df['lemmatize_text']
# docs1=preprocessed_df1['lemmatize_text']
# Nouns_docs=preprocessed_df1['nouns_verbs'].apply(lambda x:' '.join(x))

CV_countvec = CountVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=1).fit(docs)
CV_words = CV_countvec.get_feature_names()

cv_doc_embeddings = embedding_model.encode(docs)
cv_word_embeddings =embedding_model.encode(CV_words)

# cv_keywords,cv_phrases=countvec_phrase_generator(docs)
cv_keywords_doc,cv_phrases_doc=phrase_generator(docs,CV_words,CV_countvec)



## TFIDF


In [22]:
def phrase_generator(docs,words,countvec):
    
    top_n:int=10
    diversity:0.5
    df = countvec.transform(docs)
   
    keywords=[]    
    phrases=[]
    
    
    for index, doc in enumerate(docs):
        doc_words = [words[i] for i in df[index].nonzero()[1]]
        if doc_words:
            doc_word_embeddings = np.array([tfidf_word_embeddings[i] for i in df[index].nonzero()[1]])
            distances = cosine_similarity([tfidf_doc_embeddings[index]], doc_word_embeddings)[0]
            doc_keywords = [(doc_words[i], round(float(distances[i]), 4)) for i in distances.argsort()[-top_n:]]
            doc_keywords=','.join([key[0] for key in doc_keywords])
            doc_phrases=''.join([key[0] for key in doc_keywords])
            keywords.append(doc_keywords)
            phrases.append(doc_phrases)                                 
    return keywords,phrases
                              

In [24]:
# docs1=preprocessed_df1['lemmatize_text']
tfidf_countvec = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=1).fit(docs)
tfidf_words = tfidf_countvec.get_feature_names()

tfidf_doc_embeddings = embedding_model.encode(docs)
tfidf_word_embeddings = embedding_model.encode(tfidf_words)

tfidf_keywords_doc,tfidf_phrases_doc=phrase_generator(docs,tfidf_words,tfidf_countvec)

df=tfidf_countvec.transform(docs)
print(len(df[2].nonzero()[1]))
print(len(df[2].nonzero()[0]))
len(docs[2].split())

C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


24
24


39

In [ ]:
# sample_data['mmr_cv_keywords']=cv_keywords
# sample_data['mmr_tfidf_bigram_keywords']=tfidf_keywords
# sample_data['cv_doc_keywords']=cv_phrases_doc
# sample_data['tfidf_doc_keywords']=tfidf_phrases_doc

In [ ]:
sample_data.to_excel('../data/Keyword_validation_output_final.xlsx')
sample_data

# 3.Visualization

## KEPPLER MAPPER
### A scikit library that provides the functionality to project data visually, to cluster it, and to create connections between it.
#### source: git clone https://github.com/MLWave/kepler-mapper


### Goal : 
  1) Convert Documents into Features matrix \
  2) Phrase combine with keywords for Projection \
  3) Check keywords or phrases which are common and uncommon across all documents \
  
### Steps:

 1) List of 2D representations of each of our sentences into features using TFIDF
 2) Dimensionality Reduction: TruncatedSVD , Isomap \
 3) Cluster: DBSCAN we’re going to cluster the Phrase sentences \
 4) Common Keyphrase in Cluster : TF-IDF ngrams (1-3) to help us identify each cluster i.e.Phrases of length 3 which are most common.
 
### Benefits
1. The sentences in a cluster do have overlap in meaning (that’s why they’re clustered together)
2. To get overview of various and most important phrases with feature words that are group together 
 
 ### Analysis of Graph and Usage
 1. Node Label: Most common word or Phrase in all sentence in that cluster
 2. + Cluster details: Above and below average feature words with Members associated and with their    
 3. Links shows relation across keywords across the clusters 
 
## Takeaway Points and Further steps
### 1. As we can still see same word has associated with different cluster. 
### 2. Development with Analysis of this graph will be crucial step to bring contextual changes with our workflow in timely manner. 
### 3. Variation in input documents will be analyzed i.e. document with Noun Phrase joined to be feeded in Graph.
### 4. Sentence similarity will be Replaced <b>Universal sentence encoder</b> while taking to Development phase
### 5. BERT Pretrained model can be used for preparing vectors with development phase.
### 6. Generalized Encoder and Decoder can be Productionized which can be useful in other modules 

In [25]:
import kmapper as km
# Project all of the "sentences" down into a 2D representation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import Isomap
from sklearn.preprocessing import MinMaxScaler
from sklearn import cluster

In [26]:
def keyphrase_mapper(phrases):
    mapper = km.KeplerMapper(verbose=2)

    projected_X = mapper.fit_transform(np.array(phrases),
    projection=[TfidfVectorizer(analyzer="char",
                            ngram_range=(1,6),
                            max_df=0.85,
                            min_df=0.05),
            TruncatedSVD(n_components=100,
                         random_state=1729),
            Isomap(n_components=2,
                   n_jobs=-1)],
    scaler=[None, None, MinMaxScaler()])

    print("SHAPE",projected_X.shape)

    return projected_X


In [27]:
# Cluster the projected data
def cluster_visualization(docs,projected_X,path):
    mapper = km.KeplerMapper(verbose=2)
    graph = mapper.map(projected_X, clusterer=cluster.DBSCAN(eps=0.5, min_samples=3))
    # Get counts of phrases from data (to be used for the label and for cluster statistics)
    vec = TfidfVectorizer(analyzer="word",
                  strip_accents="unicode",
                  stop_words="english",
                  ngram_range=(1,3),
                  max_df=0.9,
                  min_df=0.02)

    interpretable_inverse_X = vec.fit_transform(docs).toarray()
    interpretable_inverse_X_names = vec.get_feature_names()

    print("SHAPE", interpretable_inverse_X.shape)
    print("FEATURE NAMES SAMPLE",
    interpretable_inverse_X_names[:400])
    html = mapper.visualize(graph,X=interpretable_inverse_X,
                        X_names=interpretable_inverse_X_names,
                        path_html=path,
                        lens=projected_X,
                        lens_names=["ISOMAP1", "ISOMAP2"],
                        title="Visualizing Text - Abstract",
                        )


In [29]:
projected_X=keyphrase_mapper(cv_phrases)
projected_tfidf_X=keyphrase_mapper(tfidf_phrases)

cluster_visualization(docs,projected_X,'Abstract_CV_mmr.html')
cluster_visualization(docs,projected_tfidf_X,'Abstract_tfidf_mmr.html')

KeplerMapper(verbose=2)
..Composing projection pipeline of length 3:
	Projections: TfidfVectorizer(analyzer='char', max_df=0.85, min_df=0.05, ngram_range=(1, 6))
		TruncatedSVD(n_components=100, random_state=1729)
		Isomap(n_jobs=-1)
	Distance matrices: False
False
False
	Scalers: None
None
MinMaxScaler()
..Projecting on data shaped (300,)

..Projecting data using: 
	TfidfVectorizer(analyzer='char', max_df=0.85, min_df=0.05, ngram_range=(1, 6))


..Created projection shaped (300, 958)
..Projecting on data shaped (300, 958)

..Projecting data using: 
	TruncatedSVD(n_components=100, random_state=1729)

..Projecting on data shaped (300, 100)

..Projecting data using: 
	Isomap(n_jobs=-1)


..Scaling with: MinMaxScaler()

SHAPE (300, 2)
KeplerMapper(verbose=2)
..Composing projection pipeline of length 3:
	Projections: TfidfVectorizer(analyzer='char', max_df=0.85, min_df=0.05, ngram_range=(1, 6))
		TruncatedSVD(n_components=100, random_state=1729)
		Isomap(n_jobs=-1)
	Distance matrices: Fals

C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


SHAPE (2000, 521)
FEATURE NAMES SAMPLE ['10', '100', '15', '20', '30', 'access', 'according', 'according invention', 'according method', 'accuracy', 'accurate', 'accurately', 'achieve', 'achieved', 'acid', 'acquiring', 'acquisition', 'action', 'active', 'added', 'adding', 'addition', 'adjacent', 'adjusted', 'adjusting', 'adjustment', 'adopted', 'adopting', 'advantage', 'agent', 'air', 'algorithm', 'analysis', 'angle', 'anti', 'apparatus', 'application', 'application thereof', 'applied', 'area', 'arranged', 'art', 'assembly', 'associated', 'automatic', 'automatically', 'avoided', 'away', 'axis', 'base', 'based', 'basis', 'battery', 'beam', 'bearing', 'belongs', 'belongs field', 'belongs technical', 'belongs technical field', 'better', 'block', 'board', 'body', 'box', 'calculating', 'calculation', 'capable', 'carbon', 'carried', 'carrying', 'caused', 'cavity', 'cell', 'center', 'central', 'certain', 'chamber', 'change', 'channel', 'characteristic', 'characterized', 'circuit', 'clamping',

C:\Users\HP\anaconda3\envs\Pytorchgpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


SHAPE (2000, 521)
FEATURE NAMES SAMPLE ['10', '100', '15', '20', '30', 'access', 'according', 'according invention', 'according method', 'accuracy', 'accurate', 'accurately', 'achieve', 'achieved', 'acid', 'acquiring', 'acquisition', 'action', 'active', 'added', 'adding', 'addition', 'adjacent', 'adjusted', 'adjusting', 'adjustment', 'adopted', 'adopting', 'advantage', 'agent', 'air', 'algorithm', 'analysis', 'angle', 'anti', 'apparatus', 'application', 'application thereof', 'applied', 'area', 'arranged', 'art', 'assembly', 'associated', 'automatic', 'automatically', 'avoided', 'away', 'axis', 'base', 'based', 'basis', 'battery', 'beam', 'bearing', 'belongs', 'belongs field', 'belongs technical', 'belongs technical field', 'better', 'block', 'board', 'body', 'box', 'calculating', 'calculation', 'capable', 'carbon', 'carried', 'carrying', 'caused', 'cavity', 'cell', 'center', 'central', 'certain', 'chamber', 'change', 'channel', 'characteristic', 'characterized', 'circuit', 'clamping',

In [ ]:
projected_X=keyphrase_mapper(cv_phrases_doc)
projected_tfidf_X=keyphrase_mapper(tfidf_phrases_doc)

cluster_visualization(docs1,projected_X,'Abstract_CV_doc.html')
cluster_visualization(docs1,projected_tfidf_X,'Abstract_tfidf_doc.html')


KeplerMapper(verbose=2)
..Composing projection pipeline of length 3:
	Projections: TfidfVectorizer(analyzer='char', max_df=0.85, min_df=0.05, ngram_range=(1, 6))
		TruncatedSVD(n_components=100, random_state=1729)
		Isomap(n_jobs=-1)
	Distance matrices: False
False
False
	Scalers: None
None
MinMaxScaler()
..Projecting on data shaped (2000,)

..Projecting data using: 
	TfidfVectorizer(analyzer='char', max_df=0.85, min_df=0.05, ngram_range=(1, 6))



In [ ]:
from kmapper import jupyter
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 64em; }</style>"))

jupyter.display(path_html="Abstract_tfidf_doc.html")


In [66]:
from kmapper import jupyter
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

jupyter.display(path_html="Abstract_tfidf_mmr.html")